#### ingest the constructor json file

In [0]:
dbutils.widgets.text("data_source","testing")
value_data_source = dbutils.widgets.get("data_source")

In [0]:
%run "../../constants/configuration"


In [0]:
%run "../../utils/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructors_df = spark.read \
    .schema(constructors_schema) \
    .json(f"{bronze_container_path}/constructors.json")

##### drop column url

In [0]:
constructors_drop_df = constructors_df.drop(constructors_df["url"])

##### rename columns and add ingestion date

In [0]:
constructor_final_df = add_ingestion_date(constructors_drop_df)

In [0]:
constructors_final_df = constructor_final_df.withColumnRenamed("constructorId", "constructor_id")\
                                              .withColumnRenamed("constructorRef", "constructor_ref")\
                                              .withColumn("data_source", lit(value_data_source))

In [0]:
constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("motor_dev.silver.constructors")